In [2]:
import csv

# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open("q6.dat").readlines()]

# write it as a new CSV file
with open("q6.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)

In [5]:
import pandas as pd
df = pd.read_csv('q6.csv')

In [90]:
from statsmodels.formula.api import ols
fit = ols('y ~ d1 + d2 + d3 -1', data=df).fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     34.98
Date:                Sat, 12 Dec 2020   Prob (F-statistic):           8.52e-16
Time:                        02:48:08   Log-Likelihood:                -47231.
No. Observations:                4266   AIC:                         9.447e+04
Df Residuals:                    4263   BIC:                         9.449e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
d1          2.288e+04    467.450     48.947      0.000     2.2e+04    2.38e+04
d2          2.508e+04    380.019     65.997      0.000    2.43e+04    2.58e+04
d3          2.797e+04    404.778     69.109      0.000    2.72e+04    2.88e+04
==============================================================================
Omnibus:                      683.592   Durbin-Watson:                   0.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1122.887
Skew:                           1.074   Prob(JB):                    1.47e-244
Kurtosis:                       4.306   Cond. No.                         1.23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
## I believe from what I have seen on https://www3.nd.edu/~rwilliam/stats2/l25.pdf het white is equivalent to the nR^2 test
import statsmodels.stats.diagnostic as sms
test = sms.het_white(fit.resid, fit.model.exog)
test

(11.720435279954744,
 0.002850623205591812,
 5.872229932042058,
 0.0028394189761031556)

In [67]:
import math
y_hat = fit.fittedvalues
err = df.y - y_hat
se = math.sqrt((err ** 2).sum()/4263)
se

15566.872946322332

In [78]:
# fit represents the original plot
import random

def get_new_test():
    new_u = []
    for i in range(4266):
        new_u.append(random.uniform(0,1)*se)
    
    new_y = []
    for i in range(4266):
        new_y.append(fit.params[0]*df.d1[i] + fit.params[1]*df.d2[i]+fit.params[2]*df.d3[i]+new_u[i])
    
    df['new_y'] = pd.Series(new_y)
    new_fit = ols('new_y ~ d1 + d2 + d3 -1', data=df).fit()
    return sms.het_white(new_fit.resid, new_fit.model.exog)[2]

In [81]:
B = 199
F_tests = []
for i in range(B):
    F_tests.append(get_new_test())

In [91]:
# caculate the p value
b_p_value = 0
for i in range(49):
    if F_tests[i]<test[2]:
        b_p_value += 1

if b_p_value/B > 0.5:
    b_p_value = 1-(b_p_value/B)
else:
    b_p_value = b_p_value/B
    
b_p_value

0.24623115577889448